In [ ]:
from sepal_ui import sepalwidgets as sw
import ipyvuetify as v

from utils import messages as ms

In [ ]:
#input and output as mutable object 
class File_input_io:
    def __init__(self):
        #input 
        self.file = None
        
        #output 
        self.values = None

fi_io = File_input_io()

In [ ]:
from scripts import file

#create the output 
fi_output = sw.Alert().add_msg(ms.FILE_INPUT)

#create the button
fi_btn = sw.Btn(ms.FILE_SELECT)

# describe the file to be use
fi_file_txt = sw.Markdown(ms.FILE_TXT)

#create the only widget 
fi_select = sw.FileInput(['.tif', '.tiff'])
fi_output.bind(fi_select, fi_io, 'file')

#create a file selctor tile
id_ = "input_widget"
title = "Select file"

fi_tile = sw.Tile(
    id_,
    title,
    btn=fi_btn,
    inputs=[fi_file_txt, fi_select],
    output=fi_output
)

In [ ]:
#create an automatic process 
from scripts import run
from functools import partial

def process_start(widget, event, data, output):
    
    #toggle the loading button 
    widget.toggle_loading()
    
    #load inputs 
    file = getattr(fi_io, 'file')
    
    #check input 
    if not output.check_input(file, ms.NO_FILE): return widget.toggle_loading()
    
    #launch the process 
    try:
        values = list(run.validate_file(file, output))
        setattr(fi_io, 'values', values)
    
        output.add_msg("the values {} have been selected".format(values), 'success')
        
        #change the dropdown values (not working in debug)
        reset_bands(values)
    
    except Exception as e: 
        output.add_live_msg(str(e), 'error')
    
    #toggle the loading button 
    widget.toggle_loading()
    
    return 

fi_btn.on_event('click', partial(
    process_start,
    output = fi_output
))

In [ ]:
fi_tile